# Run the cell bellow just once

In [85]:
#!unzip /home/coutinho/Downloads/ml-100k.zip

## Task Description

1.   O(s) usuário(s) mais crítico(s) na avaliação de filmes.   Encontrar aquele id cujas notas são, em média, as menores;

2.    O(s) filme(s) mais mal avaliado(s) pelos usuários.

3.    O(s) filme(s) mais bem avaliado(s) pelos usuários.

4.    Média de avaliação para cada gênero de filmes;

5.    Avaliação média de filmes por ano.

    -        Listar qual o ano com a melhor média de avaliação de filmes;

6.    Qual a ocupação mais propensa a dar más avaliações a filmes;

    -      Encontrar a média de avaliação para cada ocupação de usuário e mostrar os menores e maiores valores


# Helper Functions
## The task constrains the solution up to the standard python libray

- idea 1: desing a handler class to load and iterate through the dataset

In [86]:
class _Datasettype(list):
    """Wrapper mimics the functionality of Series in the pandas package. 
    
    Data struct wraps the loaded dataset into a customized container.

    Example:
        >>>ds = _Datasettype([[1,2,3,4,5,6],
                              [4,3,2,4,5,1]] 
                            )
        >>>ds[0, 2]
        >>>ds=modules._Datasettype(    [[1,2,3,4,5,6],
                                        [4,3,2,4,5,1],
                                        [4,3,2,4,5,1],
                                        [4,3,2,4,5,1],
                                        [4,3,2,4,5,1],
                                        [4,3,2,4,5,1],
                                        [4,3,2,4,5,1]])
        >>>ds[0,2]
        >>>[3]
        >>>ds[0:4,2]
        >>>[3, 2, 2, 2]
        >>>ds[0,2:4]
        >>>[[3, 4]]

        ..note:
            This wrapper implements `in`, `len(.)`, `==`, `[idx]`, `[idx1, idx2]`, `[start1:stop1, start2:stop2]`
    """
    def __init__(self, data: list):
        self._data=data
        super().__init__(self)
        
    def __contains__(self, obj):
        return [obj in "".join(x) for x in self._data]

    def __eq__(self, obj):
        if isinstance(obj, list):
            obj = list(map(lambda x: "".join(x),obj))
            return [x in obj  for x in self._data]
        return [x == obj  for x in self._data]

    def __iter__(self):
        #return iter([self[idx] for idx in range(len(self))])
        return iter(self._data)
    
    def __len__(self) -> int:
        return len(self._data)
           
    def __getitem__(self, idx: int) -> int:

        if isinstance(idx, tuple):
            _slice1=idx[0]
            _slice2=idx[1]

            if isinstance(_slice1, int):
                return _Datasettype([self._data[_slice1][_slice2]])
            if isinstance(_slice2, int):
                return _Datasettype(list(map(lambda x: x[_slice2], self._data[_slice1])))
            
                
            return _Datasettype([x[_slice2] for x in self._data[_slice1]])

        elif isinstance(idx, list):

            _selected_content=[]

            for i, boolean in enumerate(idx):
                if boolean:
                    _selected_content.append(self._data[i])

            return _Datasettype(_selected_content)
        else:    
            return _Datasettype(self._data[idx])


class DataFrame(_Datasettype):
    """Handler class trying to mimic DataFrame class

    Args:
        path (string): path to where the dataset is saved.

    Properties:
        loc (_Datatype): raw data in _Datatype type.
    
    """
    def __init__(self, path: str, separator: str, columns: list):
        
        self._path=path
        self._separator=separator
        self._columns=columns
        self._raw_data=self.load()
        self._data=self._set_header()
        
    @property
    def loc(self) -> _Datasettype:
        return self._raw_data
    
    def __len__(self) -> int:
        return len(self._raw_data)

    def _set_header(self) -> dict:
        return {key: self._raw_data[:,i] for i, key in enumerate(self._columns)}

    def load(self) -> _Datasettype:
        data=[]
        with open(self._path, encoding = "ISO-8859-1") as f:
            for line in  f.readlines():
                data.append(
                    tuple(
                            map(lambda x: x.replace('\n', ''), line.split(sep=self._separator))
                        )
                    )
        return _Datasettype(data)



def flat(dataset: list) -> list:
    return list(map(float,[x for y in dataset for x in y]))

def mean(dataset: list) -> float:
    sigma = sum(dataset)
    N = len(dataset)
    return sigma / N

def std(dataset: list) -> float:
    sqrt = lambda x: x**(1/2)
    mu = mean(dataset)
    sigma = sum([x - mu for x in dataset])
    N = len(dataset)

    return sqrt(sigma**2/(N-1))

In [87]:
header='user id | item id | rating | timestamp'.split(' | ')
dataset = DataFrame('ml-100k/u.data', '\t', header)

header = 'user id | age | gender | occupation | zip code'.split(' | ')
user = DataFrame('ml-100k/u.user', '|', header)

header ='movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Childrens | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'.split(' | ')
item = DataFrame('ml-100k/u.item', '|', header)



In [88]:
user_id = user['user id']
user_mean = { 
    u_id : mean(flat(dataset['rating'][dataset['user id']==u_id])) for u_id in user_id 
}

## Task \#1 
List the users id with mean review lesser than the dataset's mean review

## Top 5 critics 

In [89]:
sorted(user_mean.items(), key=lambda key_value: key_value[1])[0:5]

[('181', 1.4919540229885058),
 ('405', 1.8344640434192674),
 ('445', 1.9851851851851852),
 ('685', 2.05),
 ('774', 2.0580357142857144)]

## Task \#2 
List the baddest movie ratings

## Top 5 baddest ratings

In [90]:
dataset['item id'][dataset['rating']=='1'][0:5]

[['3', '7', '7'],
 ['3', '4', '6'],
 ['1', '0', '8', '1'],
 ['7', '9', '6'],
 ['2', '8', '8']]

## Task \#3 
The best movie ratings
## Top 5

In [91]:
dataset['item id'][dataset['rating']=='5'][0:5]

[['4', '6', '5'],
 ['1', '0', '1', '4'],
 ['2', '2', '2'],
 ['3', '8', '7'],
 ['9', '5']]

## Task \#4
Mean rating by genre


In [92]:
genre=['unknown',
 'Action',
 'Adventure',
 'Animation',
 'Childrens',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [93]:
gen_mean={}
for gen in genre:
    gen_movies = item['movie id'][item[gen]=='1']

    gen_mean[gen] = mean(flat(dataset['rating'][dataset['item id']==gen_movies]))

In [94]:
gen_mean

{'unknown': 3.2,
 'Action': 3.480245417953027,
 'Adventure': 3.503526503308369,
 'Animation': 3.5766990291262135,
 'Childrens': 3.3532442216652742,
 'Comedy': 3.3940734781442745,
 'Crime': 3.6322780881440098,
 'Documentary': 3.6728232189973613,
 'Drama': 3.6873793708484772,
 'Fantasy': 3.2152366863905324,
 'Film-Noir': 3.9215233698788228,
 'Horror': 3.2903893172841827,
 'Musical': 3.521396851029471,
 'Mystery': 3.63813155386082,
 'Romance': 3.621704948358255,
 'Sci-Fi': 3.5607227022780834,
 'Thriller': 3.5090069495245064,
 'War': 3.815811874866993,
 'Western': 3.6132686084142396}

## Task \#5
mean rating by year

In [95]:
years = list(set([x.split('-')[-1] for x in item['release date']]))

In [96]:
mean_year_rating={}
check_year = lambda x, y: [y in movie for movie in x]
for year in years:
    movies_by_year = item['movie id'][check_year(item['release date'],year)]
    mean_year_rating[year] = mean(flat(dataset['rating'][dataset['item id'] == movies_by_year]))

In [97]:
max(mean_year_rating.items(), key=lambda x: x[1])

('1942', 4.398467432950191)

## Task \#6
baddest rating by occupation

In [98]:
occupations = ['administrator',
'artist',
'doctor',
'educator',
'engineer',
'entertainment',
'executive',
'healthcare',
'homemaker',
'lawyer',
'librarian',
'marketing',
'none',
'other',
'programmer',
'retired',
'salesman',
'scientist',
'student',
'technician',
'writer',
]

In [99]:
occupation_mean_rating={}
for occupation in occupations:
    user_w_occupation = user['user id'][user['occupation']==occupation]

    occupation_mean_rating[occupation] = mean(flat(dataset['rating'][dataset['user id'] == user_w_occupation]))

In [100]:
print(min(occupation_mean_rating.items(),key=lambda x:x[1]), max(occupation_mean_rating.items(),key=lambda x:x[1]) )

('healthcare', 2.896219686162625) ('none', 3.779134295227525)
